# Age and Gender Prediction

In [18]:
import matplotlib.pyplot as plt
import seaborn as sb

In [19]:
from sklearn.metrics import confusion_matrix
import itertools
import tensorflow as tf
import plotly.express as px
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical # convert to one-hot-encoding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, InputLayer
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.losses import BinaryCrossentropy, SparseCategoricalCrossentropy

In [20]:
data = pd.read_csv("/kaggle/input/age-gender-and-ethnicity-face-data-csv/age_gender.csv")

In [21]:
data.head

In [22]:
data = data.drop(["img_name"],axis=1)

In [23]:
y = data.drop(["ethnicity","pixels"],axis =1)

In [24]:
x = data.drop(["ethnicity","age","gender"],axis =1)

In [25]:
x.head

In [26]:
y.head

In [27]:
y.head

In [28]:
x.head

In [29]:
x = pd.Series(x["pixels"])
x = x.apply(lambda x: x.split(' '))
x = x.apply(lambda x: np.array(list(map(lambda z: np.int(z), x))))
x = np.array(x)
x = np.stack(np.array(x), axis=0)
x = np.reshape(x, (-1, 48, 48))



In [30]:
x = x.reshape(-1,48,48,1)
print(x.shape)

In [31]:
plt.imshow(x[0])

In [32]:
y.head

In [35]:
age = data["age"]

In [ ]:
age.isnull().sum()

In [37]:
age = pd.cut(age,bins = [0,3,20,52,68,116],labels = ["0","1","2","3","4"])

In [42]:
age = np.array(age)
gender = np.array(data["gender"])

In [43]:
agef = to_categorical(age,num_classes =5)

In [44]:
agef

In [45]:
gender = np.array(gender)
genderf = to_categorical(gender,num_classes =2)

In [46]:
genderf

In [47]:
x  = x/255.0

In [48]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # dimesion reduction
        rotation_range=5,  # randomly rotate images in the range 5 degrees
        zoom_range = 0.1, # Randomly zoom image 10%
        width_shift_range=0.1,  # randomly shift images horizontally 10%
        height_shift_range=0.1,  # randomly shift images vertically 10%
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(x)

In [49]:
from sklearn.model_selection import train_test_split
x_train_gender,x_test_gender,y_train_gender,y_test_gender = train_test_split(x,genderf,test_size =0.3)
x_train_age,x_test_age,y_train_age,y_test_age = train_test_split(x,agef,test_size =0.3)

In [50]:
def my_model(num_classes, activation, loss):
    model = Sequential() 
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding = "same", input_shape=(48,48,1)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding = "same"))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding = "same"))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation= activation))
    model.compile(optimizer='Adam',
              loss= loss,
              metrics=['accuracy'])
    return model

In [51]:
early_stopping = EarlyStopping(patience=10, min_delta=0.001,restore_best_weights=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, 
                                            factor=0.5, min_lr=0.00001)

In [52]:
epochs = 200  # for better result increase the epochs
batch_size = 64

In [53]:
model_age = my_model(5,"softmax",'categorical_crossentropy')
fit_age = model_age.fit(x_train_age, y_train_age, batch_size=batch_size,
                              epochs = epochs, validation_data = (x_test_age,y_test_age), steps_per_epoch= x_train_age.shape[0] // batch_size, callbacks= [early_stopping, learning_rate_reduction])

In [54]:
model_gender = my_model(2,"softmax",'categorical_crossentropy')
fit_gender = model_gender.fit(x_train_gender, y_train_gender, batch_size=batch_size,
                              epochs = epochs, validation_data = (x_test_gender,y_test_gender), steps_per_epoch= x_train_gender.shape[0] // batch_size, callbacks= [early_stopping, learning_rate_reduction])

In [56]:
loss_age,acc_age = model_age.evaluate(x_test_age,y_test_age,verbose=0)
loss_gender,acc_gender = model_gender.evaluate(x_test_gender,y_test_gender,verbose=0)


In [57]:
print(f'This model will pridict age of a person with accuracy {acc_age} and with loss {loss_age}')
print(f'This model will pridict gender of a person with accuracy {acc_gender} and with loss {loss_gender}')

In [112]:
model_age.save("age_predict.h5")

In [111]:
model_gender.save("gender_predict.h5")